In [1]:
from datasets import Dataset
import pandas as pd
from fontTools.ttLib.tables.ttProgram import instructions
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer, DataCollatorForSeq2Seq, GenerationConfig

# from policy_crawling.datacleaning.test import tokenizer

In [2]:
# 将Json文件转换为CSV文件
df = pd.read_json('微调数据.json')
ds = Dataset.from_pandas(df)


In [3]:
ds[:3]

{'instruction': ['你是一名实体提取、意图识别和分类领域专家，请严格遵循以下任务和工作流程的指示输出结果。\n\n任务：\n1-分析用户输入问题的语义信息和语法结构。\n2-根据分析结果，将问题中的停用词、无意图词、语气词和符号剔除。\n3-检查是否存在剔除遗漏，是否存在错误剔除，并进行矫正。\n4-将保留的文本进行实体、关键词、意图词、主语词、设备词识别。\n5-识别出的词以逗号分隔。\n6-根据分析结果、实体提取结果和给出的意图分类标签进行问题的意图分类。\n7-根据输出格式输出。\n\n意图标签：商品数据，招标中标信息，政策内容，政策文件信息，日常问题。\n\n请根据分类结果与要求的输出格式对比，存在格式问题则进行矫正后输出。\n\n输出格式：\n实体识别结果：实体，实体\n意图分类：意图标签\n\n请注意输出结果严格按照输出格式进行，不需要输出分析过程。不要输出实体结果。\n\n问题：医用DR设备都有哪些品牌？',
  '你是一名实体提取、意图识别和分类领域专家，请严格遵循以下任务和工作流程的指示输出结果。\n\n任务：\n1-分析用户输入问题的语义信息和语法结构。\n2-根据分析结果，将问题中的停用词、无意图词、语气词和符号剔除。\n3-检查是否存在剔除遗漏，是否存在错误剔除，并进行矫正。\n4-将保留的文本进行实体、关键词、意图词、主语词、设备词识别。\n5-识别出的词以逗号分隔。\n6-根据分析结果、实体提取结果和给出的意图分类标签进行问题的意图分类。\n7-根据输出格式输出。\n\n意图标签：商品数据，招标中标信息，政策内容，政策文件信息，日常问题。\n\n请根据分类结果与要求的输出格式对比，存在格式问题则进行矫正后输出。\n\n输出格式：\n实体识别结果：实体，实体\n意图分类：意图标签\n\n请注意输出结果严格按照输出格式进行，不需要输出分析过程。不要输出实体结果。\n\n问题：动态三维脊柱姿态测量分析装置北京的供应商有哪些品牌？',
  '你是一名实体提取、意图识别和分类领域专家，请严格遵循以下任务和工作流程的指示输出结果。\n\n任务：\n1-分析用户输入问题的语义信息和语法结构。\n2-根据分析结果，将问题中的停用词、无意图词、语气词和符号剔除。\n3-检查是否存在剔除遗漏，是否存在错误剔除，并进行矫正。\n4-将保留的文本

In [ ]:
# 下载qwen2.5-3b模型
# from modelscope import snapshot_download
# model_dir = snapshot_download("Qwen/Qwen2.5-3B-Instruct", revision="master", cache_dir='./models')

# 加载tokenizer

In [5]:
# 加载分词器
model_dir = 'autodl-tmp/Qwen/Qwen2___5-3B-Instruct'
tokenizer = AutoTokenizer.from_pretrained(model_dir, use_fast=False, trust_remote_code=True)
tokenizer

OSError: Incorrect path_or_model_id: 'autodl-tmp/Qwen/Qwen2___5-3B-Instruct'. Please provide either the path to a local folder or the repo_id of a model on the Hub.

In [9]:
# 构建数据转编码函数
def process_func(example):
    MAX_LENGTH = 1800
    instruction = tokenizer(f"<|im_start|>system\n你是一名实体提取、意图识别和分类领域专家，请严格遵循以下任务和工作流程的指示输出结果。<|im_end|>\n<|im_start|>user\n{example['instruction']+example['input']}<|im_end|>\n<|im_start|>assistant\n", add_special_tokens=False)
    response = tokenizer(f"{example['output']}", add_special_tokens=False)
    input_ids = instruction['input_ids'] + response['input_ids'] + [tokenizer.pad_token_id]
    attention_mask = instruction['attention_mask'] + response['attention_mask'] + [1]
    labels = [-100] * len(instruction['input_ids']) + response['input_ids'] + [tokenizer.pad_token_id]

    if len(input_ids) > MAX_LENGTH:
        input_ids = input_ids[:MAX_LENGTH]
        attention_mask = attention_mask[:MAX_LENGTH]
        labels = labels[:MAX_LENGTH]
    return {
        'input_ids': input_ids,
        'attention_mask': attention_mask,
        'labels': labels
    }


# 数据格式化

In [10]:
tokenized_id = ds.map(process_func, remove_columns=ds.column_names)
tokenized_id

Map:   0%|          | 0/437 [00:00<?, ? examples/s]

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 437
})

In [11]:
tokenizer.decode(tokenized_id[0]['input_ids'])

'<|im_start|>system\n你是一名实体提取、意图识别和分类领域专家，请严格遵循以下任务和工作流程的指示输出结果。<|im_end|>\n<|im_start|>user\n你是一名实体提取、意图识别和分类领域专家，请严格遵循以下任务和工作流程的指示输出结果。\n\n任务：\n1-分析用户输入问题的语义信息和语法结构。\n2-根据分析结果，将问题中的停用词、无意图词、语气词和符号剔除。\n3-检查是否存在剔除遗漏，是否存在错误剔除，并进行矫正。\n4-将保留的文本进行实体、关键词、意图词、主语词、设备词识别。\n5-识别出的词以逗号分隔。\n6-根据分析结果、实体提取结果和给出的意图分类标签进行问题的意图分类。\n7-根据输出格式输出。\n\n意图标签：商品数据，招标中标信息，政策内容，政策文件信息，日常问题。\n\n请根据分类结果与要求的输出格式对比，存在格式问题则进行矫正后输出。\n\n输出格式：\n实体识别结果：实体，实体\n意图分类：意图标签\n\n请注意输出结果严格按照输出格式进行，不需要输出分析过程。不要输出实体结果。\n\n问题：医用DR设备都有哪些品牌？<|im_end|>\n<|im_start|>assistant\n实体识别结果：医用DR设备，品牌\n意图分类：商品数据<|endoftext|>'

In [15]:
# 需要过滤掉-100的位置既是输入的文本信息
tokenizer.decode(list(filter(lambda x:x!=-100, tokenized_id[0]['labels'])))

'实体识别结果：医用DR设备，品牌\n意图分类：商品数据<|endoftext|>'

# 加载半精度模型

In [25]:
import torch
model = AutoModelForCausalLM.from_pretrained(model_dir,
                                             trust_remote_code=True,
                                             # device_map='auto',
                                             torch_dtype=torch.bfloat16)
model

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(151936, 2048)
    (layers): ModuleList(
      (0-35): 36 x Qwen2DecoderLayer(
        (self_attn): Qwen2SdpaAttention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=True)
          (k_proj): Linear(in_features=2048, out_features=256, bias=True)
          (v_proj): Linear(in_features=2048, out_features=256, bias=True)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (rotary_emb): Qwen2RotaryEmbedding()
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear(in_features=2048, out_features=11008, bias=False)
          (up_proj): Linear(in_features=2048, out_features=11008, bias=False)
          (down_proj): Linear(in_features=11008, out_features=2048, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen2RMSNorm()
        (post_attention_layernorm): Qwen2RMSNorm()
      )
    )
    (norm): Qwen2RMSNorm()
  )
  (lm_head): Lin

In [26]:
model.enable_input_require_grads()  # 开启梯度检查点时，要执行该方法

In [27]:
model.dtype

torch.bfloat16

# 定义LoraConfig

In [28]:
from peft import LoraConfig, TaskType, get_peft_model
config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    # 需要训练的模型层的名字，主要就是attention部分的层，
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    inference_mode=False,   # 训练模式
    r=8,  # Lora 秩
    lora_alpha=32,  # Lora alpha 控制了 LoRA 变换的缩放因子
    lora_dropout=0.1,  # Lora dropout
)
config

LoraConfig(peft_type=<PeftType.LORA: 'LORA'>, auto_mapping=None, base_model_name_or_path=None, revision=None, task_type=<TaskType.CAUSAL_LM: 'CAUSAL_LM'>, inference_mode=False, r=8, target_modules={'q_proj', 'v_proj', 'up_proj', 'down_proj', 'gate_proj', 'k_proj', 'o_proj'}, lora_alpha=32, lora_dropout=0.1, fan_in_fan_out=False, bias='none', use_rslora=False, modules_to_save=None, init_lora_weights=True, layers_to_transform=None, layers_pattern=None, rank_pattern={}, alpha_pattern={}, megatron_config=None, megatron_core='megatron.core', loftq_config={}, use_dora=False, layer_replication=None, runtime_config=LoraRuntimeConfig(ephemeral_gpu_offload=False))

In [29]:
model = get_peft_model(model, config)
config

LoraConfig(peft_type=<PeftType.LORA: 'LORA'>, auto_mapping=None, base_model_name_or_path='C:/Users/admin/Desktop/kedaxunfei/qwen2.5-3b数据集构建/models/Qwen/Qwen2.5-3B-Instruct', revision=None, task_type=<TaskType.CAUSAL_LM: 'CAUSAL_LM'>, inference_mode=False, r=8, target_modules={'q_proj', 'v_proj', 'up_proj', 'down_proj', 'gate_proj', 'k_proj', 'o_proj'}, lora_alpha=32, lora_dropout=0.1, fan_in_fan_out=False, bias='none', use_rslora=False, modules_to_save=None, init_lora_weights=True, layers_to_transform=None, layers_pattern=None, rank_pattern={}, alpha_pattern={}, megatron_config=None, megatron_core='megatron.core', loftq_config={}, use_dora=False, layer_replication=None, runtime_config=LoraRuntimeConfig(ephemeral_gpu_offload=False))

In [30]:
model.print_trainable_parameters()

trainable params: 14,966,784 || all params: 3,100,905,472 || trainable%: 0.4827


# 配置训练参数

In [31]:
args = TrainingArguments(
    output_dir='./output/Qwen3b_instruct_lora',
    # batch_size
    per_device_train_batch_size=1,
    # 梯度累加，如果你的显存比较小，那可以把 batch_size 设置小一点，梯度累加增大一些
    gradient_accumulation_steps=4,
    # 多少步，输出一次log
    logging_steps=1,
    # 顾名思义 epoch
    num_train_epochs=1,
    save_steps=10,
    learning_rate=1e-4,
    save_on_each_node=True,
    # 梯度检查，这个一旦开启，模型就必须执行model.enable_input_require_grads()
    gradient_checkpointing=True
)

In [32]:
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_id,
    data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer, padding=True)
)

In [33]:
# 禁用内存高效的张量收缩算法,这个函数通常用于改善大规模矩阵乘法操作的内存使用效率，特别是在使用 GPU 时。
torch.backends.cuda.enable_mem_efficient_sdp(False)


In [34]:
# 开始训练
trainer.train()

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
F:\Anaconda3\envs\qwen\lib\site-packages\torch\utils\checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
F:\Anaconda3\envs\qwen\lib\site-packages\transformers\models\qwen2\modeling_qwen2.py:688: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:263.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


Step,Training Loss


RuntimeError: 
            Some tensors share memory, this will lead to duplicate memory on disk and potential differences when loading them again: [{'base_model.model.model.embed_tokens.weight', 'base_model.model.lm_head.weight'}].
            A potential way to correctly save your model is to use `save_model`.
            More information at https://huggingface.co/docs/safetensors/torch_shared_tensors
            

# 加载 lora 权重推理
## 合并加载模型